### 优点
交叉验证可以验证模型多次，减少了模型误差中的偏差，验证集精度更加可信。<br>
交叉验证可以得到多个模型，在测试集上可以进行多次预测，增加预测结果的多样性。

## 常见错误
### 错误1：选择错误的折数
k需要人工设置，一般设置为5。K值越小，模型训练次数越少，但模型验证集精度的偏差更大。K值越大，模型训练的次数越多，需要更多的计算量，但模型验证集的偏差更小。但K值极少数会大于10。

### 错误2：数据分布不同
训练集和验证集分布应保持一致。建议用StratifiedKFold代替KFold。<br>
> KFold：最基本的K折交叉验证，不受class和group的影响。<br>
> StratifiedKFold：根据数据集的**类别**占比分布来划分训练集和验证集，使得划分后的数据集的类别占比和原始数据集近似。<br>
> GroupKFold: **保证同一个group的数据不会同时出现在训练集和验证集上**。因为如果训练集中包含了每个group的几个样例，可能训练得到的模型能够足够灵活地从这些样例中学习到特征，在验证集上也会表现很好。但一旦遇到一个新的group它就会表现很差。
> StratifiedGroupKfold：**要求数据集划分时既要考虑类别占比大致不变，又要保证同一个group的数据不能同时出现在训练集和验证集上**。

In [12]:
import numpy as np
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)
print(kf.get_n_splits(X))  # 2折
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

2
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1] TEST: [2 3]


In [14]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index] # 同一类别的样本同时出现在训练集和验证集上

TRAIN: [1 3] TEST: [0 2]
TRAIN: [0 2] TEST: [1 3]


In [15]:
from sklearn.model_selection import GroupKFold
 
X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
 
gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
    print("%s %s" % (train, test)) # 同一个组的样本不会同时出现在训练集和验证集上

[0 1 2 3 4 5] [6 7 8 9]
[0 1 2 6 7 8 9] [3 4 5]
[3 4 5 6 7 8 9] [0 1 2]


In [1]:
from sklearn.model_selection import StratifiedGroupKFold # 导入报错更新下：conda update scikit-learn
X = list(range(18))
y = [1] * 6 + [0] * 12
groups = [1, 2, 3, 3, 4, 4, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 6, 6]
sgkf = StratifiedGroupKFold(n_splits=3)
for train, test in sgkf.split(X, y, groups=groups):
    print("%s %s" % (train, test))

[ 0  2  3  4  5  6  7 10 11 15 16 17] [ 1  8  9 12 13 14]
[ 0  1  4  5  6  7  8  9 11 12 13 14] [ 2  3 10 15 16 17]
[ 1  2  3  8  9 10 12 13 14 15 16 17] [ 0  4  5  6  7 11]


### 错误3：数据划分后采样

对验证集不进行任何采样，因为验证集本身是验证模型的精度，用来反应模型的泛化能力。如果采样是必不可少的一步，还是推荐**先采样再划分验证集**，这样至少训练集和验证集是同分布的。

In [ ]:
kfold = KFold(n_splits=n_splits)
scores = []
for train,valid in kfold.split(data): # 原数据先划分训练和验证
        train_oversampled = oversample_function(train) # 对训练数据做上采样，验证集不动
        score = train_and_validate(train_oversampled,valid)
        scores.append(score)

### 错误4：过拟合验证集
特征工程中将训练集和测试集拼接在一起做处理，再计算标签编码，然后划分验证集，会泄露标签的信息，导致验证集精度虚高。**正确的做法是先将数据集划分训练集和验证集，在训练集上计算标签编码，然后在验证集上进行映射。不应该使用验证集进行任何的特征提取和转换过程。**

### 错误5：乱序的时序划分
推荐用**TimeSeriesSplit**。在时序特征中，滞后特征是非常有效的一类，但也经常会泄露验证集信息。**滞后特征应该在划分验证集之后，再进行操作**。

### 错误6：数据划分的随机性
在划分数据集时可以考虑**设置随机种子**，然后就可以固定数据划分的逻辑。

In [ ]:
SEEDS = [1, 2, 3, 4, 5]
ScoreMetric = []

for seed in SEEDS:
    seed_all(seed) # 设置所有seed
    kfold = KFold(n_splits=5, random_state=seed)
    scores = []
    for train,valid in kfold.split(data):
        score = train_and_validate(train,valid)
        scores.append(score)

    ScoreMetric.append(scores)